# Forming clusters


Initial idea is to cluster all the stands according to their similarity and then to form own surrogate for every cluster. That of course rises somes questions:
- what is the similarity measure?
- what is good cluster size?
- how do use surrogates in the end?

Because clusters and then also surrogates must be created automatically we need some way to measure and compare different clusterings and surrogates. Best way to do that would be feeding the right away in to the optimization and then see how they compare between each other and the solution in the paper. Then we just would need the optimization procedure first.

## Hierarchical clustering

In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist

In [2]:
data_dir = os.path.join(os.getcwd(), '../boreal_data')

carbon = pd.read_csv(os.path.join(data_dir, 'Carbon_storage.csv'))
HA = pd.read_csv(os.path.join(data_dir, 'Combined_HA.csv'))
deadwood = pd.read_csv(os.path.join(data_dir, 'Deadwood_volume.csv'))
revenue = pd.read_csv(os.path.join(data_dir, 'Timber_revenues.csv'))

In [3]:
X1 = carbon.copy()
X1[carbon.isnull()] = np.nanmin(carbon.values) - 1

In the following cosine metric is used, because we want to ignore the size of stands and prefer their similarity in different ways

In [4]:
Z100 = linkage(X1[:100], metric='cosine')

In [5]:
c100, coph_dists = cophenet(Z100, pdist(X1[:100]))
c100

Cophenet distance is quite close to 1 so there is no need to be worried (?)

In [6]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15,6)
plt.figure()
dendrogram(Z100)
plt.show()

Okay, this works with small data


Now question: What is this green cluster?!

In [7]:
from scipy.cluster.hierarchy import fcluster
clusters = fcluster(Z100, 0.14,criterion='distance')
clusters

In [8]:
carbon[:100][clusters==1]

In [9]:
carbon[:100][clusters==2]

In [10]:
carbon[:100][clusters==3]

That looks quite great already!

There is just one problem: now we assigned Nan-values to be smalles value - 1. This means Nan:s are not a big difference when compared to other values in the dataset -> 

In [11]:
np.nanmin(carbon.values)

In [12]:
np.nanmax(carbon.values)

As we can see, differences between 'valid' data points are much greater than between 'valid' points and points with Nan-values. So it would make sense to assign much different values for Nan:s. That would also automatically connect all the Nan-including lines to the same clusters. Of course another option is to run this clustering separately for all the lines with Nan:s and all the lines without Nan:s. I am just not sure if assigning greatly different values is more efficient or more general than doing this separately. This should be studied!

## Timing

Let's compare some timings with different sized datasets and clustering methods

### Hierarchical clustering

In [13]:
%%time
Z100 = linkage(X1[:100], metric='cosine')

In [14]:
%%time
Z1000 = linkage(X1[:1000], metric='cosine')

In [15]:
%%time
Z10000 = linkage(X1[:10000], metric='cosine')

In [16]:
%%time
Z20000 = linkage(X1[:20000], metric='cosine')

In [17]:
%%time
Zall = linkage(X1, metric='cosine')

In [18]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15,9)
plt.figure()
dendrogram(Zall, truncate_mode='lastp', p=50)
plt.show()

In [19]:
from scipy.cluster.hierarchy import fcluster
clusters_all = fcluster(Zall, 50 ,criterion='maxclust')
clusters_all

In [20]:
ind = 1
print(len(carbon[clusters_all==ind]))
carbon[clusters_all==ind][:10]


In [21]:
29666*0.35

It was said that 35% of stands were simulated. Would all stands belonging to the first cluster be those?

#### Let's also try hierarchical clustering by assigning much worse values for Nan.s

In [22]:
X2 = carbon.copy()
X2[carbon.isnull()] = np.nanmin(carbon.values) - np.nanmax(carbon.values)

In [23]:
Zall_diff = linkage(X2, metric='cosine')

In [24]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15,9)
plt.figure()
dendrogram(Zall_diff, truncate_mode='lastp', p=50)
plt.show()

In [25]:
from scipy.cluster.hierarchy import fcluster
clusters_diff = fcluster(Zall_diff, 50 ,criterion='maxclust')
clusters_diff

In [26]:
ind = 49
print(len(carbon[clusters_diff==ind]))
carbon[clusters_diff==ind][:10]


In [27]:
def compare_stands(id1, id2):
    return (carbon[clusters_diff==49].iloc[id2,0]/carbon[clusters_diff==49].iloc[id1,0],
            carbon[clusters_diff==49].iloc[id2,1]/carbon[clusters_diff==49].iloc[id1,1],
            carbon[clusters_diff==49].iloc[id2,5]/carbon[clusters_diff==49].iloc[id1,5])

In [28]:
inside_variances = [var(compare_stands(0,i)) for i in range(1,len(carbon[clusters_diff==49]))]

In [29]:
print('Maximum variance: {},\nminimun variance: {},\nmedian variance: {}'.format(max(inside_variances), min(inside_variances), np.median(inside_variances)))

In [30]:
inside_variances_all = [var(compare_stands(0,i)) for i in range(1,len(carbon[clusters_all==1]))]

In [31]:
print('Maximum variance: {},\nminimun variance: {},\nmedian variance: {}'.format(max(inside_variances), min(inside_variances), np.median(inside_variances)))

In [32]:
ind = 5568
((clusters_all==1)[:ind]==(clusters_diff==49)[:ind]).all()

In [33]:
ind = 6653
((clusters_all==1)[ind:]==(clusters_diff==49)[ind:]).all()

In [34]:
((clusters_all==1)==(clusters_diff==49)).all()

## K-means

In [35]:
from scipy.cluster.vq import kmeans,vq

In [36]:
%%time
data100 = X1[:100]
centroids100, _ = kmeans(data100, 50)
idx100,  _ = vq(data100, centroids100)

In [37]:
%%time
data1000 = X1[:1000]
centroids1000, _ = kmeans(data1000, 50)
idx1000,  _ = vq(data1000, centroids1000)

In [38]:
%%time
data10000 = X1[:10000]
centroids10000, _ = kmeans(data10000, 50)
idx10000,  _ = vq(data10000, centroids10000)

In [39]:
%%time
data_all = X1
centroidsall, _ = kmeans(data_all, 50)
idxall,  _ = vq(data_all, centroidsall)

Problem is, it is not possible to define cosine distance using scipy or sklearn. Luckily I found this one kmeans implementation from the web: https://stackoverflow.com/questions/5529625/is-it-possible-to-specify-your-own-distance-function-using-scikit-learn-k-means

In [40]:
from kmeans import kmeans, randomsample

In [41]:
%%time
randomcenters = randomsample(data_all.values, 50)
centers, xtoc, dist = kmeans(data_all.values, randomcenters, delta=.001, maxiter=100, metric='cosine', verbose=2)

## Timing results


K-means is remarkably faster than hierarchical clustering. K-means's weakness is the need to know number of clusters before hand. Anyway if I don't really have to replicate the data and its real structure, I can just assign as big number of clusters as possible to still solve MILP program. If the number is big enough the error will be small enough to get relatively reliable results.